# Django Tutorial (part 4)
Back to the project home:

In [1]:
cd "$HOME/workspace/python-course/Level 4/01 Django"

In [2]:
cd workspace/mysite
tree -I __pycache__

.
├── db.sqlite3
├── manage.py
├── mysite
│   ├── asgi.py
│   ├── __init__.py
│   ├── settings.py
│   ├── urls.py
│   └── wsgi.py
└── tennis
    ├── admin.py
    ├── apps.py
    ├── __init__.py
    ├── migrations
    │   ├── 0001_initial.py
    │   └── __init__.py
    ├── models.py
    ├── templates
    │   └── tennis
    │       └── file3.html
    ├── tests.py
    ├── urls.py
    └── views.py

6 directories, 17 files


Restart the server:

In [55]:
fuser -k 7000/tcp
python manage.py runserver 7000 &

[4] 49221


Add some more views:

In [103]:
cat << EOF > tennis/views.py
from django.http import HttpResponse
from django.shortcuts import render
from .models import Match
from .forms import InputForm

def index(request)odels_and:
    return HttpResponse("Tennis index")

# Create your views here.
def homeView(request):
    context = {}
    context['form']= InputForm()
    return render(request, "home.html", context)

# Create your views here.
def filter(request, team):
    hi = request.GET.get('hi', 'default')
    lo = request.GET.get('lo', 'default')
    return HttpResponse(f"Team {team}<br>hi={hi}<br>lo={lo}")

def results(request, team):
    response = f"You're looking at the results for team: {team}"
    return HttpResponse(response)

def vote(request, question_id):
    return HttpResponse("You're voting on question %s." % question_id)

def f1(request, param1, param2):
    return HttpResponse(f"f1 {param1} {param2}")

def f2(request):
    if request.method == 'POST': # If the form has been submitted...
        form = InputForm(request.POST) # A form bound to the POST data
        if form.is_valid(): # All validation rules pass
            first_name = request.POST.get('first_name', 'default')
            last_name = request.POST.get('last_name', 'default')
            
        return HttpResponse(f'''
            first_name:{first_name}<br>
            last_name:{last_name}<br>
            ''')
    else:
        form = ContactForm() # An unbound form
    return render_to_response('contact.html', {
        'form': form,
    })

def processForm(request):
    from django import forms
    class NameForm(forms.Form):
        your_name = forms.CharField(label="Your name", max_length=100)
    
    return HttpResponse(f"processForm {request.path}")
    
def f3(request):
    context = {'name':'Chris', 'color':'blue', 'text': 'this is text from the context'}
    return render(request, "tennis/file3.html", context)
EOF

Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
November 30, 2024 - 19:59:40
Django version 5.1.3, using settings 'mysite.settings'
Starting development server at http://127.0.0.1:7000/
Quit the server with CONTROL-C.

[30/Nov/2024 19:59:56] "POST /tennis/f2/ HTTP/1.1" 200 15
cat << EOF > tennis/views.py^J


We'll investigate the `context` parameter below.  
Wire these new views into the tennis.urls module by adding the following path() calls

In [59]:
cat << EOF > tennis/urls.py
from django.urls import path

from . import views

urlpatterns = [
    # ex: /
    path('', views.index, name='index'),
    # ex: /filter/

    #firefox 'http://localhost:7000/tennis/filter/Blue?a=9&b=3'
    path('filter/<team>/', views.filter, name='filter'),

    # ex: /results/
    path('<str:team>/results/', views.results, name='results'),
    # ex: /tennis/5/vote/
    path('<int:question_id>/vote/', views.vote, name='vote'),
    path('f1/<param1>/<param2>/', views.f1, name='f1'),

    # firefox 'http://localhost:7000/tennis/f2/'
    path('f2/', views.f2, name='f2'),
    
    # firefox 'http://localhost:7000/tennis/processForm/'
    path('home/', views.homeView, name='homeView'),
    
    # firefox 'http://localhost:7000/tennis/f3/'
    path('f3/', views.f3, name='f3'),
]
EOF

/home/chris/workspace/python-course/src/38 Django/workspace/mysite/tennis/views.py changed, reloading.
/home/chris/workspace/python-course/src/38 Django/workspace/mysite/tennis/views.py changed, reloading.
Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
November 30, 2024 - 17:08:45
Django version 5.1.3, using settings 'mysite.settings'
Starting development server at http://127.0.0.1:7000/
Quit the server with CONTROL-C.

Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
Error: That port is already in use.
cat << EOF > tennis/urls.py^J
[3]-  Exit 1                  python manage.py runserver 7000


Take a look in your browser of some urls

In [60]:
# the tennis index
firefox http://localhost:7000/tennis

/home/chris/workspace/python-course/src/38 Django/workspace/mysite/tennis/urls.py changed, reloading.
Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
November 30, 2024 - 17:08:50
Django version 5.1.3, using settings 'mysite.settings'
Starting development server at http://127.0.0.1:7000/
Quit the server with CONTROL-C.

# the tennis index^J


In [11]:
# using a filter with extra path info
firefox 'http://localhost:7000/tennis/filter/Blue?hi=9&lo=3'

[30/Nov/2024 16:44:54] "GET /tennis HTTP/1.1" 301 0
[30/Nov/2024 16:44:54] "GET /tennis/ HTTP/1.1" 200 12


add info in the request

In [12]:
firefox 'http://localhost:7000/tennis/Blue/results/'

[30/Nov/2024 16:45:01] "GET /tennis/filter/Blue?hi=9&lo=3 HTTP/1.1" 301 0
[30/Nov/2024 16:45:01] "GET /tennis/filter/Blue/?hi=9&lo=3 HTTP/1.1" 200 25


pass parameters in the request

In [13]:
firefox 'http://localhost:7000/tennis/f1/aaa/bbb'

[30/Nov/2024 16:45:08] "GET /tennis/Blue/results/ HTTP/1.1" 200 44


inspect the request object

In [14]:
firefox 'http://localhost:7000/tennis/f2/'

[30/Nov/2024 16:45:16] "GET /tennis/f1/aaa/bbb HTTP/1.1" 301 0
[30/Nov/2024 16:45:16] "GET /tennis/f1/aaa/bbb/ HTTP/1.1" 200 10


These urls work, but coding your output in a str is very inflexible.  In practice you should use templates.  
So first, create a template directory

In [1]:
pwd
mkdir -p tennis/templates/tennis

/home/chris/workspace/python-course/Level 4/01 Django


You can pass information to the template from the `context` parameter of the `render`method.  
Create a template file using a context:

In [61]:
cat << EOF > tennis/templates/tennis/file3.html
<h1>*** This is file3.html ***</h1>
The context contains: 
<li>color: {{ color }} </li>
<li>text: {{ text }} </li>
EOF

[30/Nov/2024 17:08:51] "GET /tennis/ HTTP/1.1" 200 12


check out the template in the browser:

In [91]:
firefox 'http://localhost:7000/tennis/f3/'

/home/chris/workspace/python-course/src/38 Django/workspace/mysite/tennis/views.py changed, reloading.
Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
November 30, 2024 - 17:35:04
Django version 5.1.3, using settings 'mysite.settings'
Starting development server at http://127.0.0.1:7000/
Quit the server with CONTROL-C.

firefox 'http://localhost:7000/tennis/f3/'^J


<h1> DONE TO HERE !!! </h1>

Create forms.py

In [82]:
cat << EOF > tennis/forms.py
from django import forms

# creating a form 
class InputForm(forms.Form):
    first_name = forms.CharField(max_length = 200, initial="Chris")
    last_name = forms.CharField(max_length = 200, initial="Seddon")
    roll_number = forms.IntegerField(
		help_text = "Enter 6 digit roll number",
        initial=523681
	)
    password = forms.CharField(widget = forms.PasswordInput())
EOF

/home/chris/workspace/python-course/src/38 Django/workspace/mysite/tennis/forms.py changed, reloading.
Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
November 30, 2024 - 17:20:28
Django version 5.1.3, using settings 'mysite.settings'
Starting development server at http://127.0.0.1:7000/
Quit the server with CONTROL-C.

cat << EOF > tennis/forms.py^J

db.sqlite3  manage.py   mysite/     tennis/     


create template

In [79]:
cat << EOF > tennis/templates/home.html
<form action = "/tennis/f2/" method = "post">
	{% csrf_token %}
	{{form }}
	<input type="submit" value=Submit>
</form>
EOF

/home/chris/workspace/python-course/src/38 Django/workspace/mysite/tennis/forms.py changed, reloading.
Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
November 30, 2024 - 17:19:31
Django version 5.1.3, using settings 'mysite.settings'
Starting development server at http://127.0.0.1:7000/
Quit the server with CONTROL-C.

cat << EOF > tennis/templates/home.html^J


In [99]:
firefox http://localhost:7000/tennis/home

/home/chris/workspace/python-course/src/38 Django/workspace/mysite/tennis/views.py changed, reloading.
Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
November 30, 2024 - 17:42:17
Django version 5.1.3, using settings 'mysite.settings'
Starting development server at http://127.0.0.1:7000/
Quit the server with CONTROL-C.

firefox http://localhost:7000/tennis/home^J


<h1> DONE TO HERE !!! </h1>

In [33]:
cat << EOF > tennis/templates/tennis/file3.html
# import the standard Django Forms
# from built-in library
from django import forms 
  
# creating a form  
class InputForm(forms.Form): 
  
    first_name = forms.CharField(max_length = 200) 
    last_name = forms.CharField(max_length = 200) 
    roll_number = forms.IntegerField( 
                     help_text = "Enter 6 digit roll number"
                     ) 
    password = forms.CharField(widget = forms.PasswordInput())
EOF
sleep 2
firefox http://localhost:7000/tennis/f3 

Add some error handling by modifying `detail()`:

In [ ]:
cat << EOF >polls/views.py
from django.http import Http404
from django.http import HttpResponse
from django.shortcuts import render
from .models import Question

def index(request):
    latest_question_list = Question.objects.order_by('-pub_date')[:5]
    context = {'latest_question_list': latest_question_list}
    return render(request, 'polls/index.html', context)
    
def detail(request, question_id):
    try:
        question = Question.objects.get(pk=question_id)
    except Question.DoesNotExist:
        raise Http404("Question does not exist")
    return render(request, 'polls/detail.html', {'question': question})
    
def results(request, question_id):
    response = "You're looking at the results of question %s."
    return HttpResponse(response % question_id)

def vote(request, question_id):
    return HttpResponse("You're voting on question %s." % question_id)    
EOF

Now add a template that will generate a 404: Page not found" error

In [ ]:
cat << EOF >polls/templates/polls/detail.html
{{ question }}
EOF

and fire up `firefox`

In [ ]:
firefox http://localhost:7000/polls/detail/34

Add a more realistic template:

In [ ]:
cat << EOF >polls/templates/polls/detail.html
<h2>This is template 'polls/detail.html'</h2>
<h2>{{ question.question_text }}</h2>
<ul>
{% for choice in question.choice_set.all %}
    <li>{{ choice.choice_text }}</li>
{% endfor %}
</ul>
EOF

Try it out:

In [ ]:
firefox http://localhost:7000/polls/

Finally, let's take a look at our project structure:

In [ ]:
tree -I __pycache__

You can find further details on the original tutorial:  
https://docs.djangoproject.com/en/3.2/intro/tutorial01/